## Question 1

In [1]:
import pandas as pd

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url)

print(len(df))

3


In [3]:
print(df[0])

    Postal Code           Borough  \
0           M1A      Not assigned   
1           M2A      Not assigned   
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
..          ...               ...   
175         M5Z      Not assigned   
176         M6Z      Not assigned   
177         M7Z      Not assigned   
178         M8Z         Etobicoke   
179         M9Z      Not assigned   

                                         Neighbourhood  
0                                         Not assigned  
1                                         Not assigned  
2                                            Parkwoods  
3                                     Victoria Village  
4                            Regent Park, Harbourfront  
..                                                 ...  
175                                       Not assigned  
176                                       Not assigned  
177                                       

In [4]:
#Select the right table
df1 = df[0]

In [5]:
#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df1.columns = ['PostalCode','Borough','Neighborhood']
print(df1.head())
print(df1.shape)

  PostalCode           Borough               Neighborhood
0        M1A      Not assigned               Not assigned
1        M2A      Not assigned               Not assigned
2        M3A        North York                  Parkwoods
3        M4A        North York           Victoria Village
4        M5A  Downtown Toronto  Regent Park, Harbourfront
(180, 3)


In [6]:
#Ignore cells with a borough that is "Not assigned"
df2 = df1[df1.Borough != 'Not assigned']
print(df2.head())
print(df2.shape)

  PostalCode           Borough                                 Neighborhood
2        M3A        North York                                    Parkwoods
3        M4A        North York                             Victoria Village
4        M5A  Downtown Toronto                    Regent Park, Harbourfront
5        M6A        North York             Lawrence Manor, Lawrence Heights
6        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
(103, 3)


In [7]:
#Combine rows with the same PostalCode value into one row with the neighborhoods separated with a comma 
df3_1 = df2.groupby('PostalCode')['Neighborhood'].apply(','.join).reset_index()
df3_2 = df2[['PostalCode','Borough']].drop_duplicates()
df3 = df3_1.join(df3_2.set_index('PostalCode'),on='PostalCode')
df3 = df3[['PostalCode','Borough','Neighborhood']]
print(df3.head())
print(df3.shape)

  PostalCode      Borough                            Neighborhood
0        M1B  Scarborough                          Malvern, Rouge
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
2        M1E  Scarborough       Guildwood, Morningside, West Hill
3        M1G  Scarborough                                  Woburn
4        M1H  Scarborough                               Cedarbrae
(103, 3)


In [8]:
#Double check that dupilicate PostalCode values have been eliminated
print(len(df3['PostalCode'].unique()))

103


In [9]:
#If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough
df3.loc[df3.Neighborhood == 'Not assigned', 'Neighborhood'] = df3.Borough
print(df3.head())
print(df3.shape)

  PostalCode      Borough                            Neighborhood
0        M1B  Scarborough                          Malvern, Rouge
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
2        M1E  Scarborough       Guildwood, Morningside, West Hill
3        M1G  Scarborough                                  Woburn
4        M1H  Scarborough                               Cedarbrae
(103, 3)


In [10]:
#In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
df4 = df3.reset_index(drop=True)
df4.shape

(103, 3)

## Question 2

In [11]:
#Geocoder didn't work
"""
pip install geocoder
import geocoder # import geocoder
for i in range(1):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(df4.loc[i+1, 'PostalCode']))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    df4.loc[i+1, 'Latitude'] = latitude
    df4.loc[i+1, 'Longitude'] = longitude

print(df4.head())
print(df4.shape)
"""

"\npip install geocoder\nimport geocoder # import geocoder\n # initialize your variable to None\nlat_lng_coords = None\n# loop until you get the coordinates\ni = 0\nwhile(lat_lng_coords is None):\n    g = geocoder.google('{}, Toronto, Ontario'.format(df4.loc[1, 'PostalCode']))\n    lat_lng_coords = g.latlng\n    i += 1\n    print(i)\nlatitude = lat_lng_coords[0]\nlongitude = lat_lng_coords[1]\ndf4.loc[i+1, 'Latitude'] = latitude\ndf4.loc[i+1, 'Longitude'] = longitude\n"

In [16]:
#Imported Geospatial_Coordinates.csv
geospatial = pd.read_csv('Geospatial_Coordinates.csv')
geospatial.columns = ['PostalCode','Latitude','Longitude']
geospatial.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
#Join dataframes
df_laglng = df4.join(geospatial.set_index('PostalCode'), on='PostalCode')
print(df_laglng.head())
print(df_laglng.shape)

  PostalCode      Borough                            Neighborhood   Latitude  \
0        M1B  Scarborough                          Malvern, Rouge  43.806686   
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535   
2        M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3        M1G  Scarborough                                  Woburn  43.770992   
4        M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476  
(103, 5)


## Question 3

In [29]:
import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


##### Define Foursquare Credentials (ID and SECRET were removed before pushing to GitHub)

In [22]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [54]:
#Use a functon to get nearby venues of given postal codes and associated latitudes and longitudes
def getNearbyVenues(PostalCode, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for code, lat, lng in zip(PostalCode, latitudes, longitudes):
                    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            code, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(df_laglng['PostalCode'], df_laglng['Latitude'], df_laglng['Longitude'], radius=500)

print(toronto_venues.shape)
toronto_venues.head()

(2128, 7)


,PostalCode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,M1C,43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
2,M1C,43.784535,-79.160497,Great Shine Window Cleaning,43.783145,-79.157431,Home Service
3,M1C,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,M1E,43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank


In [56]:
#Check how many venues were returned for each neighborhood
toronto_venues.groupby('PostalCode').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode,,,,,,
M1B,1,1,1,1,1,1
M1C,3,3,3,3,3,3
M1E,8,8,8,8,8,8
M1G,3,3,3,3,3,3
M1H,9,9,9,9,9,9
...,...,...,...,...,...,...
M9N,2,2,2,2,2,2
M9P,7,7,7,7,7,7
M9R,4,4,4,4,4,4


##### Note: Four neighborhoods appear to have no venues, which will be dropped from analysis from this point onward.

In [57]:
#Find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 270 uniques categories.


In [58]:
#Normalize the Venue Category data for clustering

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']])

# add neighborhood column back to dataframe
toronto_onehot.insert(0,'PostalCode', toronto_venues['PostalCode'])

print(toronto_onehot.shape)
toronto_onehot.head()

(2128, 271)


,PostalCode,Venue Category_Accessories Store,Venue Category_Adult Boutique,Venue Category_Airport,Venue Category_Airport Food Court,Venue Category_Airport Gate,Venue Category_Airport Lounge,Venue Category_Airport Service,Venue Category_Airport Terminal,Venue Category_American Restaurant,...,Venue Category_Turkish Restaurant,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Video Game Store,Venue Category_Vietnamese Restaurant,Venue Category_Warehouse Store,Venue Category_Wine Bar,Venue Category_Wine Shop,Venue Category_Wings Joint,Venue Category_Women's Store,Venue Category_Yoga Studio
0,M1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M1C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M1C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M1C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M1E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
toronto_grouped = toronto_onehot.groupby('PostalCode').mean().reset_index()
toronto_grouped.shape

(99, 271)

In [63]:
#Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [64]:
# add clustering labels
toronto_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_grouped.join(df_laglng.set_index('PostalCode'), on='PostalCode')

print(toronto_merged.shape)
toronto_merged.head()

(99, 276)


,Cluster Labels,PostalCode,Venue Category_Accessories Store,Venue Category_Adult Boutique,Venue Category_Airport,Venue Category_Airport Food Court,Venue Category_Airport Gate,Venue Category_Airport Lounge,Venue Category_Airport Service,Venue Category_Airport Terminal,...,Venue Category_Warehouse Store,Venue Category_Wine Bar,Venue Category_Wine Shop,Venue Category_Wings Joint,Venue Category_Women's Store,Venue Category_Yoga Studio,Borough,Neighborhood,Latitude,Longitude
0,3,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,3,M1C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,3,M1E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,3,M1G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Scarborough,Woburn,43.770992,-79.216917
4,3,M1H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Scarborough,Cedarbrae,43.773136,-79.239476


In [65]:
#Visualize the resulting clusters

# create map (latitude and longitude are for PostalCode "M1B")
map_clusters = folium.Map(location=[43.806686, -79.194353], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters